In [39]:
import os
import datetime
import string

import pandas as pd
import numpy as np

import sqlalchemy

In [3]:
# Mix all the csv into one
citibike_files = ['JC-201601-citibike-tripdata.csv', 'JC-201602-citibike-tripdata.csv', 'JC-201603-citibike-tripdata.csv', 'JC-201604-citibike-tripdata.csv',
'JC-201605-citibike-tripdata.csv', 'JC-201606-citibike-tripdata.csv', 'JC-201607-citibike-tripdata.csv',
'JC-201608-citibike-tripdata.csv', 'JC-201609-citibike-tripdata.csv', 'JC-201610-citibike-tripdata.csv', 'JC-201611-citibike-tripdata.csv',
'JC-201612-citibike-tripdata.csv']
dfs = [pd.read_csv(file) for file in citibike_files]
result = pd.concat(dfs)
result.to_csv('year-citibike-files.csv', index=False)

In [4]:
# Dataframe with all csv
year_df = pd.read_csv('year-citibike-files.csv')
print(year_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247584 entries, 0 to 247583
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Trip Duration            247584 non-null  int64  
 1   Start Time               247584 non-null  object 
 2   Stop Time                247584 non-null  object 
 3   Start Station ID         247584 non-null  int64  
 4   Start Station Name       247584 non-null  object 
 5   Start Station Latitude   247584 non-null  float64
 6   Start Station Longitude  247584 non-null  float64
 7   End Station ID           247584 non-null  int64  
 8   End Station Name         247584 non-null  object 
 9   End Station Latitude     247584 non-null  float64
 10  End Station Longitude    247584 non-null  float64
 11  Bike ID                  247584 non-null  int64  
 12  User Type                247204 non-null  object 
 13  Birth Year               228585 non-null  float64
 14  Gend

In [5]:
print(year_df.head())

   Trip Duration           Start Time            Stop Time  Start Station ID  \
0            362  2016-01-01 00:02:52  2016-01-01 00:08:54              3186   
1            200  2016-01-01 00:18:22  2016-01-01 00:21:42              3186   
2            202  2016-01-01 00:18:25  2016-01-01 00:21:47              3186   
3            248  2016-01-01 00:23:13  2016-01-01 00:27:21              3209   
4            903  2016-01-01 01:03:20  2016-01-01 01:18:24              3195   

  Start Station Name  Start Station Latitude  Start Station Longitude  \
0      Grove St PATH               40.719586               -74.043117   
1      Grove St PATH               40.719586               -74.043117   
2      Grove St PATH               40.719586               -74.043117   
3       Brunswick St               40.724176               -74.050656   
4            Sip Ave               40.730743               -74.063784   

   End Station ID End Station Name  End Station Latitude  \
0            3209   

In [7]:
print(year_df.isna().sum())

Trip Duration                  0
Start Time                     0
Stop Time                      0
Start Station ID               0
Start Station Name             0
Start Station Latitude         0
Start Station Longitude        0
End Station ID                 0
End Station Name               0
End Station Latitude           0
End Station Longitude          0
Bike ID                        0
User Type                    380
Birth Year                 18999
Gender                         0
dtype: int64


In [8]:
print(year_df.dtypes)

Trip Duration                int64
Start Time                  object
Stop Time                   object
Start Station ID             int64
Start Station Name          object
Start Station Latitude     float64
Start Station Longitude    float64
End Station ID               int64
End Station Name            object
End Station Latitude       float64
End Station Longitude      float64
Bike ID                      int64
User Type                   object
Birth Year                 float64
Gender                       int64
dtype: object


In [9]:
# change column types to datetime
year_df['Start Time'] = pd.to_datetime(year_df['Start Time'],infer_datetime_format=True)
year_df['Stop Time'] = pd.to_datetime(year_df['Stop Time'],infer_datetime_format=True)
year_df.dtypes

Trip Duration                       int64
Start Time                 datetime64[ns]
Stop Time                  datetime64[ns]
Start Station ID                    int64
Start Station Name                 object
Start Station Latitude            float64
Start Station Longitude           float64
End Station ID                      int64
End Station Name                   object
End Station Latitude              float64
End Station Longitude             float64
Bike ID                             int64
User Type                          object
Birth Year                        float64
Gender                              int64
dtype: object

In [10]:
# reset the index
year_df.reset_index(drop=True, inplace=True)

# add an ID column -- this will be useful for counts and eventually a primary key
year_df['id'] = year_df.index

In [11]:
# updating column names to remove spaces and upper case letters
year_df.columns = [x.replace(' ','_').lower() for x in year_df.columns]

In [12]:
# calculate age and check for outliers
year_df['age'] = 2016 - year_df['birth_year'].values
year_df['age'].sort_values(ascending=False)[:10]

C:\Users\blinks\AppData\Local\Temp\ipykernel_5812\2612929238.py:3: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  year_df['age'].sort_values(ascending=False)[:10]


215055    116.0
136211     82.0
164274     79.0
187190     79.0
180373     79.0
213567     79.0
177320     76.0
112336     76.0
149803     76.0
124501     75.0
Name: age, dtype: float64

In [14]:
year_df.drop([215055], inplace=True)
year_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 247583 entries, 0 to 247583
Data columns (total 17 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   trip_duration            247583 non-null  int64         
 1   start_time               247583 non-null  datetime64[ns]
 2   stop_time                247583 non-null  datetime64[ns]
 3   start_station_id         247583 non-null  int64         
 4   start_station_name       247583 non-null  object        
 5   start_station_latitude   247583 non-null  float64       
 6   start_station_longitude  247583 non-null  float64       
 7   end_station_id           247583 non-null  int64         
 8   end_station_name         247583 non-null  object        
 9   end_station_latitude     247583 non-null  float64       
 10  end_station_longitude    247583 non-null  float64       
 11  bike_id                  247583 non-null  int64         
 12  user_type       

In [16]:
# investigate the missing values on columns birth year
missing_birth_year = year_df[year_df[['birth_year']].isnull().any(axis=1)]
missing_birth_year.head()

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,birth_year,gender,id,age
4,903,2016-01-01 01:03:20,2016-01-01 01:18:24,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,-74.051789,24573,Customer,NaN,0,4,NaN
5,883,2016-01-01 01:03:28,2016-01-01 01:18:11,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,-74.051789,24442,Customer,NaN,0,5,NaN
22,988,2016-01-01 03:16:33,2016-01-01 03:33:02,3196,Riverview Park,40.744319,-74.043991,3209,Brunswick St,40.724176,-74.050656,24662,Customer,NaN,0,22,NaN
53,3090,2016-01-01 11:07:15,2016-01-01 11:58:46,3203,Hamilton Park,40.727596,-74.044247,3203,Hamilton Park,40.727596,-74.044247,24444,Customer,NaN,0,53,NaN
57,788,2016-01-01 11:50:30,2016-01-01 12:03:39,3210,Pershing Field,40.742677,-74.051789,3195,Sip Ave,40.730743,-74.063784,24573,Customer,NaN,0,57,NaN


In [17]:
missing_birth_year['user_type'].value_counts(dropna=False)

Customer      15470
Subscriber     3529
Name: user_type, dtype: int64

In [18]:
missing_birth_year['gender'].value_counts()

0    18999
Name: gender, dtype: int64

All records with unknown genders also have missing birth year

In [19]:
missing_birth_year['user_type'].value_counts() / year_df['user_type'].value_counts()

Customer      0.996714
Subscriber    0.015232
Name: user_type, dtype: float64

99% of customers have missing birth year
1.5% of subscribers have missing birth year
All records with missing birth years also have unknown gender

In [20]:
# investigate user_type column
missing_user_type = year_df[year_df[['user_type']].isnull().any(axis=1)]
((missing_user_type['gender'] == 0) | (missing_user_type['birth_year'].isna())).sum()

0

In [21]:
missing_user_type.head()

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,birth_year,gender,id,age
25267,156,2016-03-23 09:08:34,2016-03-23 09:11:11,3214,Essex Light Rail,40.712774,-74.036486,3183,Exchange Place,40.716247,-74.033459,24444,NaN,1987.0,1,25267,29.0
25668,164,2016-03-23 22:17:45,2016-03-23 22:20:29,3183,Exchange Place,40.716247,-74.033459,3214,Essex Light Rail,40.712774,-74.036486,24675,NaN,1987.0,1,25668,29.0
25894,171,2016-03-24 11:46:39,2016-03-24 11:49:31,3214,Essex Light Rail,40.712774,-74.036486,3183,Exchange Place,40.716247,-74.033459,24697,NaN,1987.0,1,25894,29.0
26189,204,2016-03-24 20:45:45,2016-03-24 20:49:10,3183,Exchange Place,40.716247,-74.033459,3214,Essex Light Rail,40.712774,-74.036486,24387,NaN,1987.0,1,26189,29.0
26630,380,2016-03-25 19:15:56,2016-03-25 19:22:17,3183,Exchange Place,40.716247,-74.033459,3184,Paulus Hook,40.714145,-74.033552,24412,NaN,1987.0,1,26630,29.0


In [22]:
#replace NaN values with unknown
year_df.fillna({'user_type':'Unknown'},inplace=True)

In [23]:
year_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 247583 entries, 0 to 247583
Data columns (total 17 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   trip_duration            247583 non-null  int64         
 1   start_time               247583 non-null  datetime64[ns]
 2   stop_time                247583 non-null  datetime64[ns]
 3   start_station_id         247583 non-null  int64         
 4   start_station_name       247583 non-null  object        
 5   start_station_latitude   247583 non-null  float64       
 6   start_station_longitude  247583 non-null  float64       
 7   end_station_id           247583 non-null  int64         
 8   end_station_name         247583 non-null  object        
 9   end_station_latitude     247583 non-null  float64       
 10  end_station_longitude    247583 non-null  float64       
 11  bike_id                  247583 non-null  int64         
 12  user_type       

In [24]:
# creating an easier to interpret trip time column in minutes and hours
year_df['trip_minutes'] = round(year_df['trip_duration'] / 60, 2)
year_df['trip_hours'] = round(year_df['trip_duration']/(60**2),2)
year_df[['trip_duration','trip_minutes','trip_hours']].describe()

,trip_duration,trip_minutes,trip_hours
count,2.475830e+05,247583.000000,247583.000000
mean,8.856300e+02,14.760494,0.246006
std,3.593805e+04,598.967493,9.982797
min,6.100000e+01,1.020000,0.020000
25%,2.480000e+02,4.130000,0.070000
50%,3.900000e+02,6.500000,0.110000
75%,6.660000e+02,11.100000,0.180000
max,1.632981e+07,272163.470000,4536.060000


In [25]:
# i will leave the df as this, it has some unknown and some NaN values but we cant drop them
year_df.head()

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,birth_year,gender,id,age,trip_minutes,trip_hours
0,362,2016-01-01 00:02:52,2016-01-01 00:08:54,3186,Grove St PATH,40.719586,-74.043117,3209,Brunswick St,40.724176,-74.050656,24647,Subscriber,1964.0,2,0,52.0,6.03,0.10
1,200,2016-01-01 00:18:22,2016-01-01 00:21:42,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24605,Subscriber,1962.0,1,1,54.0,3.33,0.06
2,202,2016-01-01 00:18:25,2016-01-01 00:21:47,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24689,Subscriber,1962.0,2,2,54.0,3.37,0.06
3,248,2016-01-01 00:23:13,2016-01-01 00:27:21,3209,Brunswick St,40.724176,-74.050656,3203,Hamilton Park,40.727596,-74.044247,24693,Subscriber,1984.0,1,3,32.0,4.13,0.07
4,903,2016-01-01 01:03:20,2016-01-01 01:18:24,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,-74.051789,24573,Customer,NaN,0,4,NaN,15.05,0.25


In [26]:
weather_df = pd.read_csv('newark_airport_2016.csv')
weather_df.head()

,STATION,NAME,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,TSUN,WDF2,WDF5,WSF2,WSF5
0,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-01,12.75,NaN,0.0,0.0,0.0,41,43,34,NaN,270,280.0,25.9,35.1
1,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-02,9.40,NaN,0.0,0.0,0.0,36,42,30,NaN,260,260.0,21.0,25.1
2,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-03,10.29,NaN,0.0,0.0,0.0,37,47,28,NaN,270,250.0,23.9,30.0
3,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-04,17.22,NaN,0.0,0.0,0.0,32,35,14,NaN,330,330.0,25.9,33.1
4,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-05,9.84,NaN,0.0,0.0,0.0,19,31,10,NaN,360,350.0,25.1,31.1


In [27]:
weather_df.describe()

,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,TSUN,WDF2,WDF5,WSF2,WSF5
count,366.000000,0.0,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,0.0,366.000000,364.000000,366.000000,364.000000
mean,9.429973,NaN,0.104945,0.098087,0.342623,57.196721,65.991803,48.459016,NaN,217.841530,228.269231,20.484426,26.801648
std,3.748174,NaN,0.307496,1.276498,2.078510,17.466981,18.606301,17.135790,NaN,102.548282,97.415777,6.848390,8.882610
min,2.460000,NaN,0.000000,0.000000,0.000000,8.000000,18.000000,0.000000,NaN,10.000000,10.000000,6.900000,10.100000
25%,6.765000,NaN,0.000000,0.000000,0.000000,43.000000,51.250000,35.000000,NaN,150.000000,150.000000,15.000000,19.900000
50%,8.720000,NaN,0.000000,0.000000,0.000000,56.000000,66.000000,47.000000,NaN,240.000000,260.000000,19.900000,25.100000
75%,11.410000,NaN,0.030000,0.000000,0.000000,74.000000,83.000000,64.000000,NaN,300.000000,300.000000,23.900000,31.100000
max,22.820000,NaN,2.790000,24.000000,20.100000,89.000000,99.000000,80.000000,NaN,360.000000,360.000000,48.100000,66.000000


In [28]:
# drop unnecesary values
weather_df.drop(['STATION','NAME','PGTM','TSUN', 'WDF2', 'WDF5', 'WSF2', 'WSF5'],axis=1, inplace=True)

In [29]:
weather_df.dtypes

DATE     object
AWND    float64
PRCP    float64
SNOW    float64
SNWD    float64
TAVG      int64
TMAX      int64
TMIN      int64
dtype: object

In [30]:
# change date datatype to datetime
weather_df['DATE'] = pd.to_datetime(weather_df['DATE'],yearfirst=True)
weather_df.dtypes

DATE    datetime64[ns]
AWND           float64
PRCP           float64
SNOW           float64
SNWD           float64
TAVG             int64
TMAX             int64
TMIN             int64
dtype: object

In [31]:
weather_df.describe()

,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN
count,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000
mean,9.429973,0.104945,0.098087,0.342623,57.196721,65.991803,48.459016
std,3.748174,0.307496,1.276498,2.078510,17.466981,18.606301,17.135790
min,2.460000,0.000000,0.000000,0.000000,8.000000,18.000000,0.000000
25%,6.765000,0.000000,0.000000,0.000000,43.000000,51.250000,35.000000
50%,8.720000,0.000000,0.000000,0.000000,56.000000,66.000000,47.000000
75%,11.410000,0.030000,0.000000,0.000000,74.000000,83.000000,64.000000
max,22.820000,2.790000,24.000000,20.100000,89.000000,99.000000,80.000000


In [32]:
# check for nulls
weather_df.isnull().sum()

DATE    0
AWND    0
PRCP    0
SNOW    0
SNWD    0
TAVG    0
TMAX    0
TMIN    0
dtype: int64

In [34]:
# rename columns to make them easier to read
weather_df.columns = ['rec_date','avg_wind','prcp','snow_amt','snow_depth','tavg','tmax','tmin']
print(weather_df.head())

    rec_date  avg_wind  prcp  snow_amt  snow_depth  tavg  tmax  tmin
0 2016-01-01     12.75   0.0       0.0         0.0    41    43    34
1 2016-01-02      9.40   0.0       0.0         0.0    36    42    30
2 2016-01-03     10.29   0.0       0.0         0.0    37    47    28
3 2016-01-04     17.22   0.0       0.0         0.0    32    35    14
4 2016-01-05      9.84   0.0       0.0         0.0    19    31    10


In [35]:
# create new binary columns for more info
weather_df['rain'] = weather_df['prcp'].apply(lambda x: 1 if x > 0 else 0).astype(bool)
weather_df['snow'] = weather_df['snow_amt'].apply(lambda x: 1 if x > 0 else 0).astype(bool)

In [36]:
weather_df.head()

,rec_date,avg_wind,prcp,snow_amt,snow_depth,tavg,tmax,tmin,rain,snow
0,2016-01-01,12.75,0.0,0.0,0.0,41,43,34,False,False
1,2016-01-02,9.40,0.0,0.0,0.0,36,42,30,False,False
2,2016-01-03,10.29,0.0,0.0,0.0,37,47,28,False,False
3,2016-01-04,17.22,0.0,0.0,0.0,32,35,14,False,False
4,2016-01-05,9.84,0.0,0.0,0.0,19,31,10,False,False


Creating postgres tables

In [40]:
start = datetime.datetime(2016,1,1)
date_col = [start + datetime.timedelta(x) for x in range(366)]
date_df = pd.DataFrame(date_col, dtype='datetime64[ns]')
date_df.columns = ['full_date']
date_df.head()

,full_date
0,2016-01-01
1,2016-01-02
2,2016-01-03
3,2016-01-04
4,2016-01-05


In [41]:
date_df['month'] = date_df['full_date'].apply(lambda x: x.month)
date_df['day'] = date_df['full_date'].apply(lambda x: x.day)
date_df['month_name'] = date_df['full_date'].apply(lambda x: x.strftime('%B'))
date_df['day_name'] = date_df['full_date'].apply(lambda x: x.strftime('%A'))
date_df['weekend'] = date_df['day_name'].apply(lambda x: 1 if (x == 'Saturday' or x=='Sunday') else 0).astype(bool)
date_df.head()

,full_date,month,day,month_name,day_name,weekend
0,2016-01-01,1,1,January,Friday,False
1,2016-01-02,1,2,January,Saturday,True
2,2016-01-03,1,3,January,Sunday,True
3,2016-01-04,1,4,January,Monday,False
4,2016-01-05,1,5,January,Tuesday,False
